### **📊 Outpatient Analysis:**

In [67]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import scipy.stats as stats

### **Obtaining and Data Cleaning**

In [68]:
from google.colab import files
uploaded_file = files.upload()

Saving hospital_outpatient_raw.csv to hospital_outpatient_raw (3).csv


In [69]:
df = pd.read_csv("hospital_outpatient_raw.csv")

In [70]:
df.head()

,patient_id,age,sex,residence,distance_km,language,literacy_level,income_bracket,insurance_type,payer,...,diagnosis_icd10,procedure_code,labs_ordered,imaging_ordered,results_tat_days,referral_source,transport_mode,phone,email,address
0,P100000,49,Male,Urban,10.53,English,Medium,NaN,Self-pay,Self,...,M54.5,CPT87880,N,N,1.28,PrivateClinic,NaN,+254755667651,cwpusnz@gmail.com,850 Pine St
1,P100001,30,Male,Urban,5.95,Amharic,Low,Middle,Private,PrivateA,...,B34.9,CPT99213,Y,N,1.23,PHC,Bus,+254739476249,kbxosfm@gmail.com,253 Pine St
2,P100002,86,Other,Urban,0.38,Kikuyu,Low,Low,Public,PrivateB,...,B34.9,CPT80050,Y,N,0.88,Self,Motorbike,+254724972279,wgnexwh@mail.com,553 Main St
3,P100003,30,Male,Urban,19.20,English,Low,High,Public,Self,...,B34.9,CPT93000,N,N,2.46,PrivateClinic,Ambulance,254797225156,ndfipff@example.com,235 Main St
4,P100004,38,Male,Rural,3.94,Swahili,Low,Middle,Public,Self,...,E11.9,CPT71020,N,Y,2.32,Self,Motorbike,0768-559-3296,gylwobz@example.com,171 Maple St


In [71]:
df.shape

(1020, 30)

**Handling Missing Values**

In [72]:
cols = ["income_bracket","transport_mode","phone","address"]

#missingness flags signals the columns of interest
for c in cols:
    df[f"{c}_missing"] = df[c].isna().astype(int)

#fill sensible defaults and restore
df = df.fillna({"income_bracket":"Unknown","transport_mode":"Unknown","phone":np.nan,"address":np.nan})

In [73]:
# df[cols].isna().mean() * 100
# df[cols].isna().sum()
# df.loc[df[cols].isna().any(axis=1)]

**Handling Data types and Inconsistent Values:**

In [74]:
columns = ['sex','residence','language','literacy_level','income_bracket','payment_status','department']

for col in columns:
    df[col] = df[col].astype(str).str.strip().str.title().replace({"Nan":np.nan,"None":np.nan})

df['sex'] = df['sex'].replace({"F":"Female","M":"Male","Oth":"Other","femle":"Female",})
df['payment_status'] = df['payment_status'].replace({"P A I D":"Paid"})
df[["sex","department","payer","payment_status","insurance_type"]] = (df[["sex","department","payer",
                                                                          "payment_status","insurance_type"]]
                                                                      .astype('category'))

In [75]:
df['age'] = pd.to_numeric(df['age'],errors='coerce')

# mark implausible ages as NaN
df.loc[(df['age'] <= 0) | (df['age'] > 120), 'age'] = np.nan
df['age_missing_flag'] = df['age'].isna().astype(int)

In [76]:
df['encounter_date'] = pd.to_datetime(df['encounter_date'],format="%Y-%m-%d", errors="coerce")

**Remove Duplicate**

In [77]:
# Duplicate summary
dups_count = df.duplicated().sum()
print(f"Total Duplicates: {dups_count}")

dups = df.drop_duplicates(keep=False)
df = df.drop_duplicates()

Total Duplicates: 20


**Detecting outliers**

In [78]:
cols = ["age","distance_km","wait_time_min"]

df[cols] = df[cols].fillna(df[cols].median())

Q1 = df[cols].quantile(.25)
Q3 = df[cols].quantile(.75)

IQR = Q3 - Q1

In [79]:
mask_outliers = ((df[cols] < Q1 - 1.5*IQR) | (df[cols] > Q3 + 1.5*IQR)).any(axis=1)
outliers = df.loc[mask_outliers,cols]
print(f"Outliers rows detected: {outliers.shape[0]}")

Outliers rows detected: 95


In [80]:
df_clean = df.loc[~mask_outliers].copy()
print(f"Rows after dropping Outlier-rows: {df_clean.shape[0]}")

Rows after dropping Outlier-rows: 905


In [81]:
mask_inside = ((df[cols] >= Q1 - 1.5*IQR) & (df[cols] <= Q3 + 1.5*IQR)).all(axis=1)
df_clean2 = df[mask_inside]

In [82]:
if len(df) == mask_outliers.sum() + (~mask_outliers).sum():
    print("NORMAL")

NORMAL


**Save cleaned data:**

In [83]:
df_clean.to_csv("cleaned_for_eda.csv", index=False)

**What the EDA focuses on:**

- EDA reporting -> summaries & visual(percent missing, count duplicates)

- Flag -> keep them in the dataset for diagnostics, modelling and deep dives

- Wait-time pressure (median, %>60min/SLA breaches).

- No-show risk overall and by residence and income (access gap).

- Readmissions (30-day) by age band (care continuity).

- Results turnaround (diagnostics timeliness).

- Payment denials (revenue leakage).

**EDA summary report & visual(percent missing, count duplicates)**

In [84]:
# Missing values summary
cols = ["encounter_date", "phone","address"]
missing_summary = df[cols].isna().sum().to_frame("MissingCount")
missing_summary["MissingPercent"] = (df.isna().mean() * 100).round(2)
missing_summary = missing_summary.reset_index().rename(columns={"index":"Column"})

In [85]:
# Create a bar plot
fig = px.bar(missing_summary,
             x="MissingCount", y="Column",
             color="MissingPercent", orientation="h", text="MissingPercent",
             title ="Missing Values Per Column Count")

#Update layout for readeability
fig.update_layout(yaxis={"categoryorder":"total ascending"},
                  title=dict(text="Missing Values Per Column Count",
                             font=dict(size=16, family="Arial", color="black")),
                  xaxis_title=dict(text="Number of Values Missing",
                                     font=dict(size=13, family="Arial")),
                  yaxis_title=dict(text="", font=dict(size=13, family="Arial"))
            )

#Update the tracer laber for easy identification
fig.update_traces(textposition = "outside",
                 textfont=dict(size=12, family="Arial", color="black"))

#Show the plot
fig.show()

In [86]:
# Define bins and labels for wait time categories
wait_time_bins = [0, 15, 30, 60, 120]
wait_time_labels = ["0-15 min", "16-30 min", "31-60 min", "61-120 min"]

# Create wait_time_category column
df_clean["wait_time_category"] = pd.cut(
    df_clean["wait_time_min"],
    bins=wait_time_bins,
    labels=wait_time_labels,
    include_lowest=True
)

# Calculate counts and percentages for each wait time category
category_summary = (df_clean["wait_time_category"]
                   .value_counts(dropna=False, normalize=True) * 100) # Get percentages

category_summary = (category_summary
                   .reset_index()) # Reset index to ensure it's a DataFrame

# Manually rename the columns
category_summary.columns = ["Wait Time Category", "Percent"]


category_summary = (category_summary
                   .assign(Count=lambda x: (x["Percent"] * len(df_clean) / 100).astype(int)))  # Add count column


# Create horizontal bar plot with Plotly Express
fig = px.bar(
    category_summary,
    x="Percent",
    y="Wait Time Category",
    color="Wait Time Category",
    orientation="h",
    text=category_summary["Percent"].round(1).astype(str) + "%",  # Display percentages
    title="Distribution of Patient Wait Times",
    labels={"Percent": "Percentage of Encounters (%)"},
    color_discrete_sequence=px.colors.qualitative.Plotly
)

# Customize layout and traces
fig.update_layout(
    yaxis={"categoryorder": "total ascending"},
    title={"text": "Distribution of Patient Wait Times"},
    xaxis_title={"text": "Percentage of Encounters (%)", "font": {"size": 13, "family": "Arial"}},
    yaxis_title={"text": "", "font": {"size": 13, "family": "Arial"}},
    height=400,
    showlegend=False
)

fig.update_traces(
    textposition="outside",
    textfont={"size": 12, "family": "Arial", "color": "black"}
)

# Show the plot
fig.show()

In [87]:
bins = [0,15,30,60,120]
labels = ["0-15 min", "16-30 min","31-60 min","61-120 min"]

df_clean["wait_time_category"] = pd.cut(df_clean["wait_time_min"],
                                    bins = bins,
                                    labels = labels,
                                    include_lowest=True)
# department summary
depart_summary =(pd.crosstab(df_clean["department"], df_clean["wait_time_category"])).reset_index()


# Create department summary with crosstab
depart_summary = pd.crosstab(df_clean["department"], df_clean["wait_time_category"]).reset_index()

# Rename columns for clarity (fixing column names to match wait time categories)
depart_summary.columns = ["department"] + [f"{col}_count" for col in labels]

# Melt the DataFrame to long format for Plotly
depart_summary_long = pd.melt(
    depart_summary,
    id_vars="department",
    value_vars=[f"{label}_count" for label in labels],
    var_name="wait_time_category",
    value_name="count"
)

# Clean up wait_time_category column by removing '_count' suffix
depart_summary_long["wait_time_category"] = depart_summary_long["wait_time_category"].str.replace("_count", "")


In [88]:
# Create stacked bar plot with Plotly Express
fig = px.bar(
    depart_summary_long,
    x="count",
    y="department",
    orientation="h",
    color="wait_time_category",
    title="Patient Wait Time Distribution by Department",
    labels={"count": "Number of Patients",
            "wait_time_category": "Wait Time Category"},
    color_discrete_sequence=px.colors.qualitative.Plotly
)

# Update layout for better readability
fig.update_layout(
    barmode="stack",  # Stack the bars
    xaxis_title="Number of Patients",
    yaxis_title="",
    legend_title="Wait Time Category",
    showlegend=True,
    height = 600

)

# Show the plot
fig.show()

In [89]:
# Define month order for correct sorting.
month_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

#Create encounter_monthly column
df_clean["encounter_monthly"] = df_clean["encounter_date"].dt.strftime('%b')

#Calculate no-show percentages
monthly_no_show_pct = ((df_clean.groupby("encounter_monthly")["no_show"]
                        .value_counts(normalize=True)*100)
                       .rename_axis(['encounter_monthly','no_show'])
                       .reset_index(name='percent')
                       .sort_values('encounter_monthly')
                       .round({'percent':2}))

#Categorise encounter_monthly with the correct month order
monthly_no_show_pct["encounter_monthly"] = pd.Categorical(
    monthly_no_show_pct["encounter_monthly"],
    categories= month_order,
    ordered=True
)

#Sort into chronological order
monthly_no_show_pct = monthly_no_show_pct.sort_values("encounter_monthly")

In [90]:
# Create line plot
fig = px.line(
    monthly_no_show_pct,
    x='encounter_monthly',
    y='percent',
    color='no_show',
    markers=True,
    title='Monthly No-Show Percentage',
    labels={'encounter_monthly': 'Month',
            'percent': 'Percentage (%)',
            'no_show': 'No-Show Status'},
    color_discrete_sequence=px.colors.qualitative.Plotly
)

# For better readability Update layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Percentage (%)',
    legend_title='No-Show Status',
    xaxis={'tickangle': 0},
    width=1000,
    height=500
)

# Show the plot
fig.show()

In [91]:
#Overal percent missingnes by phone_missing
overal_contr = (df_clean[df_clean["no_show"] == "Y"]["phone_missing"]
               .value_counts(normalize=True)*100).reset_index(name = "percent_of_no_show").round()
overal_contr

,phone_missing,percent_of_no_show
0,0,81.0
1,1,19.0


In [92]:
df_clean['AgeGroup'] = (pd.cut(df_clean['age'],
                               bins=[5,9,14,19,35,60,100],
                               labels=['Infants','Children','Adolescents','Youths','Adults','Senior']))



# Percent readmission
readmitted_pct = ((df_clean
                   .groupby('AgeGroup',observed=False)['readmitted_30d']
                   .value_counts(normalize=True)*100)
                   .reset_index(name="percent").round()

                  )

# Create a combine bar plot
fig = px.bar(
    readmitted_pct,
    x='AgeGroup',
    y='percent',
    color='readmitted_30d',
    barmode='group',
    title='Readmission Within 30days by Age Group',
    labels={ 'percent': 'Percentage (%)'}
)
fig.show()

In [93]:
# average result TAT
result_tat = df_clean["results_tat_days"].mean().round()
print(f"Result Turn Around Time is, {result_tat} days")

Result Turn Around Time is, 2.0 days


In [94]:
# billing summary
billing_summary = (df_clean
                   .groupby(["payer","payment_status"], observed = False)
                   .agg(Count = ("payer","size"),TotalBilling = ("billing_amount","sum"))
                   .round({"TotalBilling":2})
                   .reset_index()
                   )

# Create a combine bar plot
fig = px.bar(
    billing_summary,
    x='payer',
    y='Count',
    color='payment_status',
    barmode='group',
    title='Revenue Collected',
    labels={ 'Count': 'Number of Encounters'}
)
fig.show()

In [98]:
# Create grouped table and turn index into column
df_residence = (
    df_clean
    .groupby('residence')['no_show']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={"N": "Show", "Y": "No-Show"})
    .reset_index()
)

# Melt to long form for plotting
df_residence_melted = df_residence.melt(
    id_vars="residence",
    value_vars=["Show", "No-Show"],
    var_name="Attendance",
    value_name="Count"
)

# Grouped bar plot
fig_bar = px.bar(
    df_residence_melted,
    x="residence",
    y="Count",
    color="Attendance",
    barmode="group",
    title="Show vs No-Show by Residence (grouped bar)",
    text="Count"
)
fig_bar.update_traces(textposition="outside")
fig_bar.show()
